# Python Primer: working with data

## <div style="color: #db366d"> Day 1.3 </div>

One of the best things about python is the ability to easily load, explore and manipulate data. As they say, *data is king*, and you can't have good ML outcomes without good data, just like in culinary.

![](../images/produce.jpeg)

## Loading data
Let's start by reading a text-based file without any fancy libraries.

In [ ]:
# load a file from the system into the mem space of this program
file = open("../data/gowork.csv", "r")

# extract the file contents into a variable for us to work with
file_contents = file.read()

# display the contents on the screen
print(file_contents)

However, it's really hard to work with a chunk of text like that...

Enter the `pandas` library, probably the most important Python library for data science, e.g., see the pandas' prowess below...

![](../images/rise-in-pandas.png)
<br/>
Let's try loading the same csv file with pandas below:
<br/>

In [ ]:
# import both numpy and pandas, which are often used together
# - pandas is built on top of NumPy, a lower level library
# - NumPy helps you work with large n-d arrays/matrices
import numpy as np
import pandas as pd

# importing common dataset formats is ultra simple in Python
df = pd.read_csv('../data/gowork.csv')

# the first thing after you load data is always to peek at it
print(df)

In [ ]:
# you can also display using the Jupyter Notebook's theme
df

One of the most basic things you can do is to convert a dictionary to a dataframe...

In [ ]:
import pandas as pd

handmade_dict = {
    'Breakdown' : ['Slow', 'Yes', 'Slow'],
    'Rain'      : [True, True, False],
    'Oversleep' : ['Slight', 'Major', 'Major'],
    'ML Lecture': [True, True, True]
}

# note we do not have a variable to hold the df below
pd.DataFrame(handmade_dict)

You can load a lot of common data types in here with Python and pandas.

Try the following to load a json file downloaded from data.gov.sg .

In [ ]:
# python has a built-in json library to process json
# additionally, you need some pandas json libraries 
# to load it as a pandas DataFrame
import json
from pandas.io.json import json_normalize # load a specific function

# load the file into data var
file = open('../data/datagovsg/weather.json')
data =json.load(file)

# convert raw json data into a DataFrame
df = json_normalize(data['items'],
                    record_path=['forecasts'],
                    meta=['update_timestamp',
                          ['valid_period','start'],
                          ['valid_period','end']
                         ]
                   )

# peek at the top 5 rows
df.head(5)

In [ ]:
# or peek only bottom 5 rows
df.tail(5)

## Loading data from the web
With pandas, you can load data from the web easily as well.

Try the following Star Wars web API.

In [ ]:
# we need the requests library to make *web requests*
# we will import json and pandas as usual
import requests
import json
import pandas as pd

# make a web request with the web API
# the response from the API will be store in the response var
response = requests.get('https://swapi.co/api/people')

# get the json formatted text from the response
# store it as a dictionary (dict) data structure
# dicts are common programming data structures for key-value pairs 
data_dict = response.json()

# let's peek at the dictionary first, to see what we want
data_dict

In [ ]:
# now convert the required (sub)dict into a pandas DataFrame
df = pd.DataFrame(data_dict['results'])
df

The universe of Python libraries is vast, and the general advice is to Google for an existing _approach to get your dataset_ (or anything you want to do in Python for that matter) first, before rolling your own.

For example, try the follow ultra simple way to get stocks data from the web using pandas_datareader lib.
(Note you may need to install it first by issuing `conda install pandas-datareader` command in the terminal)

In [ ]:
# pandas_datareader consolidates various popular online financial data providers
# most are free, or have have free tiers 
# some don't even require membership, like Stooq, demo'd here
import pandas_datareader.data as web

# fetch Google's stock OHLC data from Stooq and display it
df = web.DataReader('GOOG', 'stooq')
df.head(5)

#### EXERCISE: 
#### (1) Fetch current EURUSD, EURGBP, AUDUSD, USDJPY, USDCAD and NZDUSD prices from freeforexapi.com, and 
#### (2) display in a DataFrame (each currency pair should be a row of data)

In [ ]:
# TODO: write Python code to fetch the required data as instructed above


## Knowing the data
We saw a bit about how to display the loaded data. How about other information that you'd commonly need?

Try the following to obtain basic dimensionality information about datasets.

In [ ]:
# basic sizes of the dataframe
print('SHAPE of df\n---------------------')
print(df.shape)

# detailed info about the dataframe
print('\nDETAILED INFO of df\n---------------------')
df.info()

We can also get statistical descriptions of the actual data

In [ ]:
df.describe()

## Manipulating data
Now that we have the data, we will normally want to do stuff to it.

E.g., we can extract a single column...

In [ ]:
df['Close'].head()

We can then work with these columns...

In [ ]:
print('Sum of volume is', df['Volume'].sum())
print('Mean of open price is', df['Open'].mean())
print('Median of all high prices is', df['High'].median())

Slicing is a key concept when working with DataFrames. It basically means "cutting" out a piece from the entire dataset.

E.g., we can get a slice at row number (i.e., index=) 100...

In [ ]:
df.iloc[100]

A slice that contains the date "2018-12-10"...

In [ ]:
df.loc['2018-12-10']

A slice containing the 5th to 10th rows...


In [ ]:
df.iloc[5:10]

A slice containing rows between Feb and Mar:

In [ ]:
# note that the rows are sorted in reverse chronological order
df.loc['2019-01-31':'2019-01-01']

A slice containing rows from Oct onwards:

In [ ]:
df.loc['2018-10-01':]

Or slice out records that match certain conditions, e.g., a range of values or equal a certain number...

E.g., here's an alternative way to get the previous slice with date "2018-12-10"

In [ ]:
df[df.index=='2018-12-10']

In [ ]:
# find row with close price between 1000 and 1050
print('A slice that contains all rows with close price between 800 and 900')
df[(df['Close']>=800) & (df['Close']<=900)]

Note that for display purposes, we'll just leave the sliced cakes 
on the table as above, however, in most practical cases, we'll assign the sliced portion to a variable, like so:

In [ ]:
single_slice = df.iloc[100]

## Simple data cleansing

Some other common things we may want to do with the data includes re-structuring and removing stuff.

For this section let's load in the Star Wars dataset again

In [ ]:
import requests
import json
import pandas as pd

response = requests.get('https://swapi.co/api/people')
data_dict = response.json()
df = pd.DataFrame(data_dict['results'])
df

A key part of ML is knowledge representation, and part of that involves determining what to throw away.

After scrutinizing the dataset structure above, the first thing we may want to do is to remove unwanted cols...

In [ ]:
# note that the arg axis=1 tells it to drop the columns (axis=0 refers to rows)
df_new = df.drop(['created','films','homeworld','species','starships','url','vehicles'], axis=1) 
df_new

Many DataFrame manipulation operations return a new copy of the DataFrame. As shown above, the original DataFrame is untouched.

In [ ]:
df

That is, unless you specify it to be modified **in place**, like so...

In [ ]:
df.drop(['created','films','homeworld','species','starships','url','vehicles'], axis=1, inplace=True) 
df

Next, we may want to **rename** some cols to our liking...

In [ ]:
df.columns = ['birth_yr', 'updated', 'eye_col', 'sex', 'hair_col', 'height', 'weight', 'name', 'skin_col']
df

Since we're at it, might as well **re-arrange** the cols...

In [ ]:
# note that we're using self-assignment here
df = df[['name', 'height', 'weight', 'sex', 'birth_yr', 'eye_col', 'hair_col', 'skin_col', 'updated']]
df

In observing the data, it seems that there are some empty fields (e.g., hair_col), which is common especially in very large datasets.

In consequence, we may want to **replace** the non-standard empty fields ('n/a') into something more standard in Python...

In [ ]:
# NaN or 'Not a Number' is a standard way to represent nulls in pandas and numpy
import numpy as np
df.replace(['n/a', 'unknown'], np.nan, inplace=True)
df

With standard null representations, we can easily do things like viewing **where the nulls are**...

In [ ]:
df.isnull()

Or checking **how many nulls in each col**...

In [ ]:
df.isnull().sum()

And then after evaluating the nulls, perhaps we may need to **remove the records with nulls**...

In [ ]:
# again, the original df is unmodified
df.dropna()

We can also choose to **drop the columns with nulls** instead...

In [ ]:
df.dropna(axis=1)

Another common operation is to remove duplicates. We don't have dupes in this data but we can artificially create some like so:

In [ ]:
duped_df = df.append(df)
duped_df

Now we can simply call the following to **eliminate all dupes**...

In [ ]:
duped_df.drop_duplicates(inplace=True)
duped_df

A big part of data cleansing is to run some arbitrary function that formats the data to a style that is easier (and often faster) for the ML algorithm to act on.

E.g., perhaps we want to **run a function on each row** that converts all dates to our preferred simplified format.

First we define the function to run on each date. This function will omit the time portion of the date string.

In [ ]:
def format_date(date):
    return date[:10]

Then we can simply apply this function on every date in each row...

In [ ]:
df['updated'] = df["updated"].apply(format_date)
df

#### EXERCISE: 
#### (1) Get ALL planets that exist in Star Wars, and 
#### (2) obtain a slice of the above data containing all planets with diameter greater than 7000

In [ ]:
# TODO: Write Python code to achieve the above


## Saving data
After doing all sorts of things to your dataset, you would surely need to save them to disk. Everything you 

As you'd expect with Python, this is a one liner:

In [ ]:
# write to a csv file
df.to_csv('output.csv')

# or if you had formatted it as a json file
# df.to_json('output.json')